In [5]:
import os
import pickle
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from matminer.featurizers.structure import DensityFeatures, GlobalSymmetryFeatures
from matminer.featurizers.composition import ElementProperty
from mp_api.client import MPRester

API_KEY = "f3qtz1d2EV47QfPtknWcFSOXTaUCzNli" 
TARGET_PROP = "formation_energy_per_atom"

# Configuration for testing vs full pipeline
TESTING = False  # Set to False for full pipeline

if TESTING:
    CACHE_FILE = "mp_data_cache_1k_test.pkl"
    BATCH_SIZE = 20
    N_TOTAL = 200
    print("=" * 50)
    print("RUNNING IN TESTING MODE (200 samples)")
    print("=" * 50)
else:
    CACHE_FILE = "mp_data_cache_20k.pkl"
    BATCH_SIZE = 100
    N_TOTAL = 10000
    print("=" * 50)
    print("RUNNING IN FULL PIPELINE MODE (10000 samples)")
    print("=" * 50)

# ================== Node类 ===================
class Node:
    def __init__(self, name, methods):
        self.name = name
        self.methods = methods
    
    def execute(self, method, params, data):
        return self.methods[method](data, **params)

# N0: Data Fetch & Feature工程（分批+进度条+缓存+API兼容性）
def fetch_and_featurize(data=None, cache=True):
    if cache and os.path.exists(CACHE_FILE):
        print(f"[N0] Load cache: {CACHE_FILE}")
        with open(CACHE_FILE, 'rb') as f:
            df = pickle.load(f)
        print(f"Loaded {len(df)} entries from cache")
    else:
        print(f"[N0] Fetching + feature engineering from MP API ({N_TOTAL} entries)...")
        elem_feat = ElementProperty.from_preset("magpie")
        dens_feat = DensityFeatures()
        sym_feat = GlobalSymmetryFeatures()
        
        dfs = []
        fetched = 0
        
        with MPRester(API_KEY) as mpr:
            try:
                # Use the search method with proper pagination
                docs = mpr.materials.summary.search(
                    fields=["material_id", "structure", "elements", "formula_pretty", TARGET_PROP],
                    chunk_size=BATCH_SIZE,
                    num_chunks=N_TOTAL // BATCH_SIZE + 1
                )
                
                # Process in batches
                batch_count = 0
                for batch_docs in tqdm(docs, desc="Processing batches", ncols=100):
                    if not isinstance(batch_docs, list):
                        batch_docs = [batch_docs]
                    
                    # Filter out docs without target property
                    valid_docs = [d for d in batch_docs if getattr(d, TARGET_PROP, None) is not None]
                    
                    if not valid_docs:
                        continue
                    
                    # Create dataframe from batch
                    df_batch = pd.DataFrame([
                        {
                            "material_id": d.material_id,
                            "structure": d.structure,
                            "elements": d.elements,
                            "formula_pretty": getattr(d, "formula_pretty", None),
                            TARGET_PROP: getattr(d, TARGET_PROP, None)
                        }
                        for d in valid_docs
                    ])
                    
                    # Remove entries without structure
                    df_batch = df_batch.dropna(subset=["structure"]).reset_index(drop=True)
                    
                    if len(df_batch) == 0:
                        continue
                    
                    # Feature engineering
                    try:
                        df_batch["composition"] = df_batch["structure"].apply(lambda s: s.composition)
                        
                        # Element features
                        elem_features = []
                        for comp in tqdm(df_batch["composition"], desc="Element features", leave=False):
                            try:
                                elem_features.append(elem_feat.featurize(comp))
                            except Exception as e:
                                if not TESTING:  # Only print errors in full mode
                                    print(f"Error in element featurization: {e}")
                                elem_features.append([np.nan] * len(elem_feat.feature_labels()))
                        
                        df_elem = pd.DataFrame(elem_features, columns=elem_feat.feature_labels())
                        
                        # Density features
                        dens_features = []
                        for struct in tqdm(df_batch["structure"], desc="Density features", leave=False):
                            try:
                                dens_features.append(dens_feat.featurize(struct))
                            except Exception as e:
                                if not TESTING:  # Only print errors in full mode
                                    print(f"Error in density featurization: {e}")
                                dens_features.append([np.nan] * len(dens_feat.feature_labels()))
                        
                        df_dens = pd.DataFrame(dens_features, columns=dens_feat.feature_labels())
                        
                        # Symmetry features
                        sym_features = []
                        for struct in tqdm(df_batch["structure"], desc="Symmetry features", leave=False):
                            try:
                                sym_features.append(sym_feat.featurize(struct))
                            except Exception as e:
                                if not TESTING:  # Only print errors in full mode
                                    print(f"Error in symmetry featurization: {e}")
                                sym_features.append([np.nan] * len(sym_feat.feature_labels()))
                        
                        df_sym = pd.DataFrame(sym_features, columns=sym_feat.feature_labels())
                        
                        # Combine all features
                        df_feat = pd.concat([
                            df_batch.reset_index(drop=True), 
                            df_elem.reset_index(drop=True), 
                            df_dens.reset_index(drop=True), 
                            df_sym.reset_index(drop=True)
                        ], axis=1)
                        
                        dfs.append(df_feat)
                        fetched += len(df_feat)
                        
                        print(f"Processed batch {batch_count + 1}, fetched {fetched}/{N_TOTAL} entries")
                        
                        if fetched >= N_TOTAL:
                            break
                            
                    except Exception as e:
                        print(f"Error processing batch {batch_count}: {e}")
                        continue
                    
                    batch_count += 1
                
            except Exception as e:
                print(f"Error in API call: {e}")
                return None
        
        if not dfs:
            print("No data was successfully fetched!")
            return None
        
        # Combine all batches
        df = pd.concat(dfs, axis=0, ignore_index=True)
        
        # Ensure we don't exceed N_TOTAL
        if len(df) > N_TOTAL:
            df = df.iloc[:N_TOTAL].copy()
        
        print(f"Total entries fetched: {len(df)}")
        
        # Save to cache
        with open(CACHE_FILE, 'wb') as f:
            pickle.dump(df, f)
        print(f"Data saved to cache: {CACHE_FILE}")
    
    # Fe / non-Fe split
    def is_fe(x):
        if isinstance(x, list):
            return 'Fe' in x
        try:
            if isinstance(x, str):
                return 'Fe' in eval(x)
            return 'Fe' in x
        except Exception:
            return False
    
    fe_mask = df['elements'].apply(is_fe)
    df_fe = df[fe_mask].reset_index(drop=True)
    df_non_fe = df[~fe_mask].reset_index(drop=True)
    
    print(f"Non-Fe materials: {len(df_non_fe)}")
    print(f"Fe-containing materials: {len(df_fe)}")
    
    # In testing mode, ensure we have enough data for both splits
    if TESTING:
        if len(df_non_fe) < 50:
            print("Warning: Very few non-Fe materials for training in testing mode")
        if len(df_fe) < 10:
            print("Warning: Very few Fe materials for testing in testing mode")
    
    return {'train_data': df_non_fe, 'test_data': df_fe, 'full_data': df}

# N1: Impute Node
def impute_mean(data, **params):
    imp = SimpleImputer(strategy='mean')
    X_train = imp.fit_transform(data['X_train'])
    X_val = imp.transform(data['X_val'])
    X_test = imp.transform(data['X_test']) if data['X_test'] is not None else None
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'imputer': imp}

def impute_knn(data, n_neighbors=5, **params):
    # In testing mode, use fewer neighbors if needed
    if TESTING and data['X_train'].shape[0] < n_neighbors:
        n_neighbors = max(1, data['X_train'].shape[0] - 1)
    
    imp = KNNImputer(n_neighbors=n_neighbors)
    X_train = imp.fit_transform(data['X_train'])
    X_val = imp.transform(data['X_val'])
    X_test = imp.transform(data['X_test']) if data['X_test'] is not None else None
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'imputer': imp}

# N2: 特征矩阵生成
def feature_matrix(data):
    train_df = data['train_data']
    test_df = data['test_data']
    
    # Drop columns with missing rate > threshold
    missing_threshold = 0.8 if TESTING else 0.5  # More lenient in testing mode
    cols_drop = [c for c in train_df.columns if train_df[c].isna().mean() > missing_threshold]
    print(f"Dropping {len(cols_drop)} columns with >{missing_threshold*100}% missing values")
    
    # Select feature columns
    feat_cols = [c for c in train_df.columns 
                 if c not in ['material_id', 'structure', 'elements', 'formula_pretty', 'composition', TARGET_PROP] 
                 and c not in cols_drop]
    
    print(f"Using {len(feat_cols)} feature columns")
    
    # Convert to numeric
    X_train_full = train_df[feat_cols].apply(pd.to_numeric, errors='coerce')
    X_test_full = test_df[feat_cols].apply(pd.to_numeric, errors='coerce') if len(test_df) > 0 else None
    
    # Train/validation split
    split_ratio = 0.8
    split_idx = int(split_ratio * len(X_train_full))
    
    # Ensure minimum sizes in testing mode
    if TESTING:
        if split_idx < 10:
            split_idx = max(10, len(X_train_full) - 5)
    
    X_train, X_val = X_train_full.iloc[:split_idx], X_train_full.iloc[split_idx:]
    X_test = X_test_full
    
    print(f"Train/Val/Test split: {len(X_train)}/{len(X_val)}/{len(X_test) if X_test is not None else 0}")
    
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'feature_names': feat_cols}

# N3: Feature Selection
def no_selection(data, **params):
    return data

def variance_selection(data, var_ratio=0.01, **params):
    # In testing mode, use more lenient variance threshold
    if TESTING:
        var_ratio = 0.001
    
    selector = VarianceThreshold(threshold=var_ratio)
    X_train = selector.fit_transform(data['X_train'])
    X_val = selector.transform(data['X_val'])
    X_test = selector.transform(data['X_test']) if data['X_test'] is not None else None
    
    print(f"Variance selection: {data['X_train'].shape[1]} -> {X_train.shape[1]} features")
    
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'selector': selector}

# N4: Scaling
def scale_standard(data, **params):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(data['X_train'])
    X_val = scaler.transform(data['X_val'])
    X_test = scaler.transform(data['X_test']) if data['X_test'] is not None else None
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'scaler': scaler}

def scale_robust(data, **params):
    scaler = RobustScaler()
    X_train = scaler.fit_transform(data['X_train'])
    X_val = scaler.transform(data['X_val'])
    X_test = scaler.transform(data['X_test']) if data['X_test'] is not None else None
    return {'X_train': X_train, 'X_val': X_val, 'X_test': X_test, 'scaler': scaler}

# N5: Learner Node
def train_rf(data, n_estimators=100, max_depth=None, **params):
    # Adjust parameters for testing mode
    if TESTING:
        n_estimators = 20  # Fewer trees for faster training
        max_depth = 5      # Shallow trees to prevent overfitting with small data
    
    model = RandomForestRegressor(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        random_state=42, 
        n_jobs=-1
    )
    model.fit(data['X_train'], data['y_train'])
    y_val_pred = model.predict(data['X_val'])
    y_test_pred = model.predict(data['X_test']) if data['X_test'] is not None else None
    return {'model': model, 'y_val_pred': y_val_pred, 'y_test_pred': y_test_pred}

def train_gbr(data, n_estimators=100, learning_rate=0.1, max_depth=3, **params):
    # Adjust parameters for testing mode
    if TESTING:
        n_estimators = 20     # Fewer estimators
        learning_rate = 0.2   # Higher learning rate for faster convergence
        max_depth = 3         # Keep shallow
    
    model = GradientBoostingRegressor(
        n_estimators=n_estimators, 
        learning_rate=learning_rate, 
        max_depth=max_depth, 
        random_state=42
    )
    model.fit(data['X_train'], data['y_train'])
    y_val_pred = model.predict(data['X_val'])
    y_test_pred = model.predict(data['X_test']) if data['X_test'] is not None else None
    return {'model': model, 'y_val_pred': y_val_pred, 'y_test_pred': y_test_pred}

# ========= Assemble nodes =========
nodes = {
    'N0': Node('DataFetch', {'api': fetch_and_featurize}),
    'N1': Node('Impute', {'mean': impute_mean, 'knn': impute_knn}),
    'N2': Node('FeatureMatrix', {'default': feature_matrix}),
    'N3': Node('FeatSelect', {'none': no_selection, 'variance': variance_selection}),
    'N4': Node('Scale', {'std': scale_standard, 'robust': scale_robust}),
    'N5': Node('Learner', {'rf': train_rf, 'gbr': train_gbr}),
}

# ========= 主 pipeline =========
def run_pipeline(config=None):
    start = time.time()
    
    # Node0: fetch + featurize
    print("=" * 50)
    print(f"Starting ML Pipeline ({'TESTING' if TESTING else 'FULL'} mode)")
    print("=" * 50)
    
    n0_out = nodes['N0'].execute('api', {}, None)
    if n0_out is None:
        print("Failed to fetch data!")
        return None
    
    train_df = n0_out['train_data']
    test_df = n0_out['test_data']
    
    if len(train_df) == 0:
        print("No training data available!")
        return None
    
    # Check minimum data requirements
    if TESTING and len(train_df) < 20:
        print(f"Warning: Only {len(train_df)} training samples in testing mode")
    
    # Node2: 特征矩阵
    print("\n" + "=" * 30)
    print("Feature Matrix Generation")
    print("=" * 30)
    n2_out = nodes['N2'].execute('default', {}, {'train_data': train_df, 'test_data': test_df})
    
    # y split
    y_train_full = train_df[TARGET_PROP].values
    y_test_full = test_df[TARGET_PROP].values if len(test_df) > 0 else None
    
    split_ratio = 0.8
    split_idx = int(split_ratio * len(y_train_full))
    
    # Ensure minimum sizes in testing mode
    if TESTING:
        if split_idx < 10:
            split_idx = max(10, len(y_train_full) - 5)
    
    y_train, y_val = y_train_full[:split_idx], y_train_full[split_idx:]
    
    print(f"Training samples: {len(y_train)}")
    print(f"Validation samples: {len(y_val)}")
    if y_test_full is not None:
        print(f"Test samples: {len(y_test_full)}")
    
    # Node1: impute
    print("\n" + "=" * 30)
    print("Imputation")
    print("=" * 30)
    n1_out = nodes['N1'].execute('mean', {}, {
        'X_train': n2_out['X_train'], 
        'X_val': n2_out['X_val'], 
        'X_test': n2_out['X_test']
    })
    
    # Node3: feature selection
    print("\n" + "=" * 30)
    print("Feature Selection")
    print("=" * 30)
    selection_method = 'none' if TESTING else 'none'  # Keep simple for testing
    n3_out = nodes['N3'].execute(selection_method, {}, n1_out)
    
    # Node4: scaling
    print("\n" + "=" * 30)
    print("Scaling")
    print("=" * 30)
    n4_out = nodes['N4'].execute('std', {}, n3_out)
    
    # Node5: learner
    print("\n" + "=" * 30)
    print("Model Training")
    print("=" * 30)
    n4_out['y_train'] = y_train
    n4_out['y_val'] = y_val
    n4_out['y_test'] = y_test_full
    
    # Use RF for both modes, but with different params
    n5_out = nodes['N5'].execute('rf', {}, n4_out)
    
    # Metrics
    mae_val = mean_absolute_error(y_val, n5_out['y_val_pred'])
    r2_val = r2_score(y_val, n5_out['y_val_pred'])
    
    if y_test_full is not None and n5_out['y_test_pred'] is not None:
        mae_test = mean_absolute_error(y_test_full, n5_out['y_test_pred'])
        r2_test = r2_score(y_test_full, n5_out['y_test_pred'])
    else:
        mae_test, r2_test = None, None
    
    print("\n" + "=" * 30)
    print("Results")
    print("=" * 30)
    print(f"Validation MAE: {mae_val:.3f}")
    print(f"Validation R²: {r2_val:.3f}")
    
    if mae_test is not None:
        print(f"Test MAE: {mae_test:.3f}")
        print(f"Test R²: {r2_test:.3f}")
    
    print(f"\nTotal pipeline time: {time.time() - start:.1f}s")
    
    return {
        'mae_val': mae_val, 
        'r2_val': r2_val, 
        'mae_test': mae_test, 
        'r2_test': r2_test, 
        'model': n5_out['model'],
        'train_size': len(y_train),
        'val_size': len(y_val),
        'test_size': len(y_test_full) if y_test_full is not None else 0,
        'mode': 'TESTING' if TESTING else 'FULL'
    }

if __name__ == "__main__":
    print(f"Running pipeline in {'TESTING' if TESTING else 'FULL'} mode...")
    print(f"Target samples: {N_TOTAL}")
    print(f"Cache file: {CACHE_FILE}")
    print("-" * 50)
    
    results = run_pipeline()
    if results:
        print("\n" + "=" * 50)
        print("PIPELINE COMPLETED SUCCESSFULLY!")
        print("=" * 50)
        print(f"Mode: {results['mode']}")
        print(f"Data sizes - Train: {results['train_size']}, Val: {results['val_size']}, Test: {results['test_size']}")
        print(f"Performance - Val MAE: {results['mae_val']:.3f}, Val R²: {results['r2_val']:.3f}")
        if results['mae_test'] is not None:
            print(f"Test performance - MAE: {results['mae_test']:.3f}, R²: {results['r2_test']:.3f}")
    else:
        print("\n" + "=" * 50)
        print("PIPELINE FAILED!")
        print("=" * 50)

RUNNING IN FULL PIPELINE MODE (10000 samples)
Running pipeline in FULL mode...
Target samples: 10000
Cache file: mp_data_cache_20k.pkl
--------------------------------------------------
Starting ML Pipeline (FULL mode)
[N0] Load cache: mp_data_cache_20k.pkl
Loaded 10000 entries from cache
Non-Fe materials: 10000
Fe-containing materials: 0

Feature Matrix Generation
Dropping 0 columns with >50.0% missing values
Using 140 feature columns
Train/Val/Test split: 8000/2000/0
Training samples: 8000
Validation samples: 2000

Imputation


D:\Anaconda\lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['crystal_system']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\impute\_base.py:558: UserWarning: Skipping features without any observed values: ['crystal_system']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(



Feature Selection

Scaling

Model Training

Results
Validation MAE: 0.174
Validation R²: 0.883

Total pipeline time: 23.6s

PIPELINE COMPLETED SUCCESSFULLY!
Mode: FULL
Data sizes - Train: 8000, Val: 2000, Test: 0
Performance - Val MAE: 0.174, Val R²: 0.883


Pipeline started...
[N0] Fetching + feature engineering from MP API (20000 entries)...


C:\Users\48103\AppData\Roaming\Python\Python38\site-packages\mp_api\client\mprester.py:193: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/21000 [00:00<?, ?it/s]

Processing batches:   1%|▍                                      | 245/21000 [00:03<05:13, 66.22it/s]


TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'